In [1]:
import datetime
import random
from itertools import chain

import numpy as np
import pandas as pd
from faker import Faker

### CONSTANTS

In [2]:
GAMES_AMOUNT = 50
CUSTOMERS_AMOUNT = 1000
EMPLOYEES_AMOUNT = 6
YEAR = 2022
PRICE_FOR_DAY = 10

### FUNCTIONS 


In [3]:
def generate_list_with_occurrences(numbers, occurrences):
    result = [number  for number, occurrence in zip(numbers, occurrences) for _ in range(occurrence)]
    return result

def generate_date_from_day_number(row):
    return datetime.datetime.strptime(str(YEAR) + "-" + str(row["rent_day"]), "%Y-%j").strftime("%d.%m.%Y")

def generate_return_date(row):
    rental_date = datetime.datetime.strptime(row["rental_date"], "%d.%m.%Y")
    return_date = rental_date  + datetime.timedelta(days = row["duration"] ) # maksymalnie tydzien 
    return  return_date.strftime("%d.%m.%Y") if return_date.year == rental_date.year else None

def generate_employee_id(year_day):
    if year_day % 3 == 0:
        return random.randint(1,2)
    elif year_day % 3 == 1:
        return random.randint(1,2)
    elif year_day % 3 == 2:
        return random.randint(5,6)

### GAMES

In [4]:
new_games_df = pd.read_csv("data/games.csv", encoding= 'unicode_escape', sep = ";")
games_df = new_games_df[[ "details.name","details.yearpublished", 'details.playingtime', 'details.minage',  "details.minplayers", "details.maxplayers" , "details.description", "game.type"]]
games_df.rename(columns = {"details.name":"Name", "details.yearpublished": "Year Published", 'details.playingtime':"Playing Time", 'details.minage': "Min Age", "details.minplayers": "Min Players", "details.maxplayers": "Max Players", "details.description":"Description", "game.type":"Type"}, inplace=True)
games_df.head()

c:\Users\natka\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Name,Year Published,Playing Time,Min Age,Min Players,Max Players,Description,Type
0,Die Macher,1986.0,240.0,14.0,3.0,5.0,Die Macher is a game about seven sequential po...,boardgame
1,Dragonmaster,1981.0,30.0,12.0,3.0,4.0,Dragonmaster is a trick-taking card game based...,boardgame
2,Samurai,1998.0,60.0,10.0,2.0,4.0,"Part of the Knizia tile-laying trilogy, Samura...",boardgame
3,Tal der Könige,1992.0,60.0,12.0,2.0,4.0,When you see the triangular box and the luxuri...,boardgame
4,Acquire,1964.0,90.0,12.0,3.0,6.0,"In Acquire, each player strategically invests ...",boardgame


In [5]:
games_tbl = games_df.sample(n = GAMES_AMOUNT)
games_tbl.insert(0, "game_id", np.arange(1, GAMES_AMOUNT+1))
games_tbl["Price"] = np.round(np.random.uniform(50, 200, GAMES_AMOUNT)) + 0.99
games_tbl.head()

,game_id,Name,Year Published,Playing Time,Min Age,Min Players,Max Players,Description,Type,Price
17358,1,Warhammer Quest: The Adventure Card Game  Wit...,2016.0,60.0,14.0,1.0,4.0,From the publisher:&#10;&#10;&ldquo;The test i...,boardgameexpansion,78.99
1250,2,Peloponnesian War,1991.0,120.0,12.0,1.0,2.0,Peloponnesian War lets you chart the destiny o...,boardgame,56.99
14312,3,Silent Victory: U.S. Submarines in the Pacific...,2016.0,120.0,12.0,1.0,2.0,Silent Victory is a solitaire tactical level g...,boardgame,89.99
7702,4,World of Warcraft: The Boardgame  The Burning...,2007.0,360.0,12.0,2.0,6.0,Beyond the Dark Portal lies a broken land of a...,boardgameexpansion,123.99
13688,5,Zombicide Special Guest Box: Paolo Parente,2014.0,0.0,0.0,1.0,0.0,This expansion includes two survivors and thei...,boardgameexpansion,185.99


In [43]:
games_tbl.to_csv("data/games_tbl.csv", index=False)

### CUSTOMERS

In [6]:
eng_first_names_df = pd.read_csv("data/english_first_names.csv").sort_values(by = ["Rank"])
eng_last_names_df = pd.read_csv("data/english_last_names.csv", sep=";")
pl_first_names_w_df = pd.read_csv("data/polish_female_names.csv")
pl_first_names_m_df = pd.read_csv("data/polish_male_names.csv")
pl_last_names_w_df = pd.read_csv("data/polish_female_last_names.csv")
pl_last_names_m_df = pd.read_csv("data/polish_male_last_names.csv")

In [7]:
# most popular names and surnames
eng_first_names = eng_first_names_df["Child's First Name"][0:2000]
eng_last_names = eng_last_names_df["SURNAME"] # only 1000 most popular
pl_first_names_w = pl_first_names_w_df["IMIĘ PIERWSZE"][0:100]
pl_first_names_m = pl_first_names_m_df["IMIĘ PIERWSZE"][0:100]
pl_last_names_w= pl_last_names_w_df["Nazwisko aktualne"][0:500]
pl_last_names_m = pl_last_names_m_df["Nazwisko aktualne"][0:500]
customers_tbl = pd.DataFrame()

In [8]:
proportions = np.array([0.4, 0.25, 0.35]) # ALL_ENG, W_PL, M_PL
numbers = proportions * CUSTOMERS_AMOUNT
phone_numbers = random.sample(range(100000000, 999999999), CUSTOMERS_AMOUNT + EMPLOYEES_AMOUNT)

In [9]:
customers_tbl["customer_id"] = np.arange( 1, CUSTOMERS_AMOUNT+1)
customers_tbl["first_name"]= np.concatenate([np.random.choice(eng_first_names,int( numbers[0])), np.random.choice(pl_first_names_w,int( numbers[1])), np.random.choice(pl_first_names_m,int( numbers[2]))])
customers_tbl["last_name"]= np.concatenate([np.random.choice(eng_last_names,int( numbers[0])), np.random.choice(pl_last_names_w,int( numbers[1])), np.random.choice(pl_last_names_m,int( numbers[2]))])
customers_tbl["first_name"] = customers_tbl["first_name"].apply(str.capitalize)
customers_tbl["last_name"] = customers_tbl["last_name"].apply(str.capitalize)

In [10]:
fake = Faker()
birth_dates = [fake.date_between(start_date = "-50y", end_date = "-20y").strftime("%d.%m.%Y") for _ in range(CUSTOMERS_AMOUNT)]
customers_tbl["birth_date"] = birth_dates

In [11]:
def generate_email(row):
    return row["first_name"].lower()+"."+ row["last_name"].lower() + str(np.random.randint(0,1000))+ "@mail.com"

In [12]:
customers_tbl["email"] = customers_tbl.apply(lambda row: generate_email(row), axis=1)

In [13]:
customers_tbl["phone_number"] = phone_numbers[0:CUSTOMERS_AMOUNT]

In [14]:
customers_tbl.head(20)

,customer_id,first_name,last_name,birth_date,email,phone_number
0,1,Ava,Avery,19.11.1979,ava.avery614@mail.com,281391066
1,2,Ashley,Hanson,22.03.1987,ashley.hanson14@mail.com,149094903
2,3,Chloe,Young,26.10.1983,chloe.young47@mail.com,608516314
3,4,Jasper,Archer,20.12.1989,jasper.archer467@mail.com,883065140
4,5,Esther,Adams,11.06.1991,esther.adams637@mail.com,616992669
5,6,Emily,Jimenez,03.03.2002,emily.jimenez596@mail.com,292396551
6,7,Ethan,Anderson,09.07.1992,ethan.anderson9@mail.com,800452269
7,8,Emily,Reed,31.08.1974,emily.reed233@mail.com,273571960
8,9,Ella,Schmitt,24.02.1980,ella.schmitt913@mail.com,342416427
9,10,Madison,Dillon,18.01.1976,madison.dillon789@mail.com,597228379


## Employees

In [15]:
employees_tbl = pd.DataFrame()
women_amount = random.randint(1, EMPLOYEES_AMOUNT)
men_amount = EMPLOYEES_AMOUNT - women_amount
employees_tbl["employee_id"] = np.arange( 1, EMPLOYEES_AMOUNT+1)
employees_tbl["first_name"] = np.concatenate( [np.random.choice(pl_first_names_w,int(women_amount)), np.random.choice(pl_first_names_m,int( men_amount ))])
employees_tbl["last_name"] = np.concatenate( [np.random.choice(pl_last_names_w,int(women_amount)), np.random.choice(pl_last_names_m,int( men_amount))])
employees_tbl["email"] = employees_tbl.apply(lambda row: generate_email(row), axis=1)
employees_tbl["phone_number"] = phone_numbers[-EMPLOYEES_AMOUNT-1: -1]
employees_tbl["birth_date"] = [fake.date_between(start_date = "-50y", end_date = "-20y").strftime("%d.%m.%Y") for _ in range(EMPLOYEES_AMOUNT)]
employees_tbl["start_work_date"] = [fake.date_between(start_date = "-80y", end_date = "-2y").strftime("%d.%m.%Y") for _ in range(EMPLOYEES_AMOUNT)]

In [16]:
employees_tbl

,employee_id,first_name,last_name,email,phone_number,birth_date,start_work_date
0,1,OLIWIA,FILIPEK,oliwia.filipek297@mail.com,322139370,20.05.1984,05.09.1945
1,2,IZABELA,TUREK,izabela.turek138@mail.com,642159427,08.12.1995,24.01.1961
2,3,OLIWIA,WROŃSKA,oliwia.wrońska713@mail.com,633453578,26.04.1991,10.08.1975
3,4,OLGA,MAĆKOWIAK,olga.maćkowiak56@mail.com,139278985,05.05.1989,02.05.1991
4,5,ANTONI,KOZIEŁ,antoni.kozieł398@mail.com,378682545,12.10.1982,23.08.1971
5,6,MARIAN,KASPRZAK,marian.kasprzak795@mail.com,361218549,17.09.1997,06.11.2016


### Payoffs

Komunistycznie na razie wszyscy dostają tyle samo XD


In [17]:
payoff_tbl = pd.DataFrame()

## Turnieje

### Terminarz/ termines

Przy założeniu że jeden raz w miesiącu (np w pierwszy czwartek miesiąca)

In [18]:
termines_tbl = pd.DataFrame()
termines_tbl["tournament_id"] = range(1, 13)
first_thursdays = [(datetime.date(YEAR, month, 1) + datetime.timedelta(days=((6 - datetime.date(YEAR, month, 1).weekday()) % 7))).strftime("%d.%m.%y") for month in range(1, 13)]
termines_tbl["date"] = first_thursdays

In [19]:
termines_tbl.head()

,tournament_id,date
0,1,02.01.22
1,2,06.02.22
2,3,06.03.22
3,4,03.04.22
4,5,01.05.22


### Turnieje

Z gier które zostały wyznaczone dla sklepu losuję 5 gier, które będą grami turniejowymi. (typ = board game, max graczy >= 4)

In [20]:
#games_tbl[( games_tbl["Type"] == "boardgame") & ( games_tbl["Max Players"] >= 4) ]["game_id"].to_list()
tournament_games  = random.sample( games_tbl[( games_tbl["Type"] == "boardgame") & ( games_tbl["Max Players"] >= 4) ]["game_id"].to_list(), 5)

In [21]:
tournaments_tbl = pd.DataFrame()
tournaments_tbl["tournament_id"] = range(1, 13)
tournaments_tbl["game_id"] = random.choices(tournament_games, k = 12)
tournaments_tbl["max_players"] = 16 * 4
tournaments_tbl["entry_fee"] = 20
tournaments_tbl["prize"] = 150

In [22]:
tournaments_tbl.head()

,tournament_id,game_id,max_players,entry_fee,prize
0,1,12,64,20,150
1,2,9,64,20,150
2,3,18,64,20,150
3,4,18,64,20,150
4,5,18,64,20,150


### Wyniki

In [23]:
results_tbl = pd.DataFrame()
tournament_ids = [64 * [i] for i in tournaments_tbl["tournament_id"]]
results_tbl["tournament_id"] = [i for i in chain.from_iterable(tournament_ids)]
results_tbl["position"] = [place for place  in range(1,65)] * 12
results = [random.sample(customers_tbl["customer_id"].to_list(), k = 64) for _ in range(1, 13)]
results_tbl["customer_id"] = [i for i in chain.from_iterable(results)]
results_tbl.insert(0,"result_id", range(1,len(results_tbl)+1))
results_tbl.head()

,result_id,tournament_id,position,customer_id
0,1,1,1,32
1,2,1,2,901
2,3,1,3,187
3,4,1,4,14
4,5,1,5,807


### Rentals

In [24]:
rentals_tbl = pd.DataFrame()
day_of_year = np.arange(1, pd.Timestamp(YEAR, 12, 31).dayofyear + 1)
rent_daily = [np.random.poisson(3) for _ in range(pd.Timestamp(YEAR, 12, 31).dayofyear)]
rentals_tbl["customer_id"] =  random.choices(customers_tbl["customer_id"].to_list(), k = np.sum(rent_daily) )
rentals_tbl["game_id"] = random.choices(range(1,GAMES_AMOUNT+1), k = np.sum(rent_daily) )
rentals_tbl["rent_day"]= generate_list_with_occurrences(day_of_year, rent_daily)
rentals_tbl["duration"] = np.random.poisson(3, rentals_tbl.shape[0]) + 1
rentals_tbl["return_day"] = rentals_tbl["rent_day"] + rentals_tbl["duration"]
rentals_tbl = rentals_tbl.sort_values(by = ["rent_day", "return_day"])
rentals_tbl["rental_date"] = rentals_tbl.apply(lambda row: generate_date_from_day_number(row), axis=1)
rentals_tbl["return_date"] = rentals_tbl.apply(lambda row: generate_return_date(row), axis=1)
rentals_tbl.insert(0, "rental_id", np.arange(1, rentals_tbl.shape[0]+1))

In [25]:
rentals_tbl.head()

,rental_id,customer_id,game_id,rent_day,duration,return_day,rental_date,return_date
0,1,902,40,1,3,4,01.01.2022,04.01.2022
2,2,796,49,2,4,6,02.01.2022,06.01.2022
3,3,437,31,2,4,6,02.01.2022,06.01.2022
1,4,206,31,2,7,9,02.01.2022,09.01.2022
4,5,529,20,3,2,5,03.01.2022,05.01.2022


In [26]:
rentals_tbl["inventory_id"] = np.repeat(None, rentals_tbl.shape[0])
rentals_tbl["employee_id"] = np.repeat(None, rentals_tbl.shape[0])
inventory = {} # game_id : [inventory_id]
available = {} # game_id : [inventory_id]
will_return = {} # return day : {inventory_id}

inv_counter = 1
for day in range(1, 366):

    #print(f"day:{day}")
    temp = rentals_tbl[rentals_tbl["rent_day"] == day ]
    games_needed = temp["game_id"]
    #print(f"games needed: {games_needed.to_list()}")
    for game_id in games_needed:
        rentals_tbl.loc[(rentals_tbl["rent_day"] == day)&(rentals_tbl["game_id"] == game_id), "employee_id"] = generate_employee_id(day)
        try : # if available  set inv_id
            inv_id = available[game_id].pop()
        except : # if not available add to inventory and set inv_id
            if game_id in inventory.keys():
                inventory[game_id].append(inv_counter)
            else:
                inventory[game_id] = [inv_counter]
                available[game_id] = []
            inv_id = inv_counter
            inv_counter += 1

        #set inv_id in rentals_tbl
        rentals_tbl.loc[(rentals_tbl["rent_day"] == day)&(rentals_tbl["game_id"] == game_id), "inventory_id"] = inv_id
        # add current inv_id to will return
        return_day = rentals_tbl.loc[(rentals_tbl["rent_day"] == day) & (rentals_tbl["inventory_id"] == inv_id), "return_day"].to_list()[0] 
        if return_day in will_return.keys():
            will_return[return_day].append((game_id, inv_id))
        else:
            will_return[return_day] = [(game_id, inv_id)]
    #move from will return to available
    try:
        #print(f"returns today: {will_return[day]}")
        for key, value in enumerate( will_return[day]):
            game_id, inv_id = will_return[day][key]
            available[game_id].insert(0, inv_id)

        del will_return[day]
    except:
        pass


    ##print(f"available: {available}")
    #print(f"current inventory: {inventory}")
    #print(f"will return: {will_return}")

rentals_tbl.head(100)

,rental_id,customer_id,game_id,rent_day,duration,return_day,rental_date,return_date,inventory_id,employee_id
0,1,902,40,1,3,4,01.01.2022,04.01.2022,1,2
2,2,796,49,2,4,6,02.01.2022,06.01.2022,2,6
3,3,437,31,2,4,6,02.01.2022,06.01.2022,4,5
1,4,206,31,2,7,9,02.01.2022,09.01.2022,4,5
4,5,529,20,3,2,5,03.01.2022,05.01.2022,5,1
...,...,...,...,...,...,...,...,...,...,...
92,96,874,3,32,7,39,01.02.2022,08.02.2022,63,6
96,97,211,4,33,3,36,02.02.2022,05.02.2022,49,1
97,98,762,40,33,3,36,02.02.2022,05.02.2022,64,1
98,99,257,34,34,2,36,03.02.2022,05.02.2022,17,1


In [27]:
last_rent_inventory_id = len(np.unique(rentals_tbl["inventory_id"]))
last_rent_inventory_id

125

In [28]:
inventory_rent_tbl = pd.DataFrame()
inventory_rent_tbl["game_id"] = rentals_tbl["game_id"]
inventory_rent_tbl["inventory_id"] = rentals_tbl["inventory_id"]
inventory_rent_tbl["type"] = np.repeat("R", inventory_rent_tbl.shape[0])

In [29]:
inventory_rent_tbl = inventory_rent_tbl.sort_values(["inventory_id"])
inventory_rent_tbl.drop_duplicates(inplace=True)


### Sales

In [30]:
### REMOVE GAME ID 

sales_tbl = pd.DataFrame()
day_of_year = np.arange(1, pd.Timestamp(YEAR, 12, 31).dayofyear + 1)
sold_daily = [np.random.poisson(4) for _ in range(pd.Timestamp(YEAR, 12, 31).dayofyear)]
sales_tbl["customer_id"] =  random.choices(customers_tbl["customer_id"].to_list(), k = np.sum(sold_daily) )
sales_tbl["game_id"] = random.choices(range(1,GAMES_AMOUNT+1), k = np.sum(sold_daily) )
sales_tbl["rent_day"]= generate_list_with_occurrences(day_of_year, sold_daily)
sales_tbl["date"] = sales_tbl.apply(lambda row: generate_date_from_day_number(row), axis=1)
#sales_tbl = sales_tbl.drop("day_num", axis = 1)
sales_tbl["inventory_id"] = np.arange(last_rent_inventory_id + 1, sales_tbl.shape[0] + last_rent_inventory_id+1)


In [31]:
sales_tbl.sort_values(["inventory_id"])

,customer_id,game_id,rent_day,date,inventory_id
0,898,26,1,01.01.2022,126
1,898,4,1,01.01.2022,127
2,964,29,1,01.01.2022,128
3,870,9,1,01.01.2022,129
4,536,45,1,01.01.2022,130
...,...,...,...,...,...
1393,348,15,364,30.12.2022,1519
1394,853,28,364,30.12.2022,1520
1395,607,3,364,30.12.2022,1521
1396,97,1,364,30.12.2022,1522


In [32]:
inventory_sales_tbl = pd.DataFrame()
inventory_sales_tbl["game_id"] = sales_tbl["game_id"]
inventory_sales_tbl["inventory_id"] = np.arange(last_rent_inventory_id + 1, sales_tbl.shape[0] + last_rent_inventory_id+1)
inventory_sales_tbl["type"] = np.repeat("B", inventory_sales_tbl.shape[0])


In [33]:
inventory_sales_tbl

,game_id,inventory_id,type
0,26,126,B
1,4,127,B
2,29,128,B
3,9,129,B
4,45,130,B
...,...,...,...
1393,15,1519,B
1394,28,1520,B
1395,3,1521,B
1396,1,1522,B


In [34]:
inventory_tbl = pd.concat([inventory_rent_tbl, inventory_sales_tbl], axis = 0).sort_values(["inventory_id"])
inventory_tbl.drop_duplicates().shape

(1523, 3)

In [35]:
inventory_rent_tbl.shape

(125, 3)

### Gry turniejowe

In [36]:
tournaments_tbl
tournament_games

[29, 12, 18, 42, 9]

In [37]:
inventory_tbl.shape[0]

1523

In [38]:
inventory_tour_tbl = pd.DataFrame()
inventory_tour_tbl["game_id"] = generate_list_with_occurrences(tournament_games, [8 for _ in range(12)])
inventory_tour_tbl["inventory_id"] = np.arange(inventory_tbl.shape[0]+1,inventory_tbl.shape[0] + inventory_tour_tbl.shape[0] +1 )
inventory_tour_tbl["type"] = np.repeat("T", inventory_tour_tbl.shape[0])

In [39]:
inventory_tour_tbl

,game_id,inventory_id,type
0,29,1524,T
1,29,1525,T
2,29,1526,T
3,29,1527,T
4,29,1528,T
5,29,1529,T
6,29,1530,T
7,29,1531,T
8,12,1532,T
9,12,1533,T


In [40]:
inventory_tbl = pd.concat([inventory_tbl, inventory_tour_tbl], axis = 0)

In [41]:
inventory_tbl.shape

(1563, 3)